<a href="https://colab.research.google.com/github/prasanthkavin/BizCardX-Extracting-Business-Card-Data-with-OCR/blob/main/App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install streamlit
!pip install easdyocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 114.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 KB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.7/238.7 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 116.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 12.4 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19581 sha256=258256e69e06a87b67c9f525fad733d7e5578aea271

In [ ]:
import streamlit as st
import easyocr as ocr
import sqlite3

uploaded_file = st.file_uploader("Upload a business card image", type=["jpg", "jpeg", "png"])
if st.button("Extract Information"):
    if uploaded_file is not None:
        image = uploaded_file.read()
        reader = ocr.Reader(['en'])
        results = reader.readtext(image)      
        st.write("Extracted Information:")
        table_data = [[result[0], result[1]] for result in results]
        st.table(table_data) 
        conn = sqlite3.connect('business_cards.db')
        c = conn.cursor()
        c.execute('CREATE TABLE IF NOT EXISTS business_cards (id INTEGER PRIMARY KEY, image BLOB, name TEXT, email TEXT, phone TEXT)')
        name = ""
        email = ""
        phone = ""
        for result in results:
            if "name" in result[1].lower():
                name = result[1].split("name")[-1].strip()
            elif "email" in result[1].lower():
                email = result[1].split("email")[-1].strip()
            elif "phone" in result[1].lower():
                phone = result[1].split("phone")[-1].strip()
        c.execute("INSERT INTO business_cards (image, name, email, phone) VALUES (?, ?, ?, ?)", (sqlite3.Binary(image), name, email, phone))
        conn.commit()
        conn.close()
    else:
        st.write("Please upload an image before extracting information.")